# Model error analysis notebook
For error analysis workflow of PV vs non-PV trained models. Uses the v1.0 or v2.0 master datasets to retrieve actual TOTALDEMAND values, together with supporting data such as TEMPERATURE, POWER, etc.

Expects two input experiment files of predictions (one with PV, one without PV) over an equivalent forecast date range.
Experiments files are a 2-column .CSV of format:
DATETIME                TOTALDEMAND
YYYY-MM-DD HH:MM:SS     Float

These names of these input files are specified in the PV_FILE and NO_PV_FILE variables (see below). The main datafile that is used for comparison is specified in ACTUALS_FILE

At this point, the notebook processes data on a state-by-state basis. The state being analysed must be specified in the STATE variable.

In [1]:
#!pip install darts

In [2]:
from darts import TimeSeries
from darts.metrics import mape, rmse  
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from darts.utils.statistics import plot_hist, plot_residuals_analysis, plot_acf, plot_pacf
from darts.models import NaiveSeasonal

# Enable multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

ImportError: cannot import name 'TimeSeries' from 'darts' (unknown location)

In [ ]:
# Directory where the combined master project datafile is
DATA_PATH = "../../data/UNSW-PROJECT-DATA/"

#Filename of the combined master project file with the actual values
ACTUALS_FILE = "demand_pv_temp_forecast_timefeatures_hols.csv"

# Directory where the .CSV experimantal result files are
EXPERIMENT_PATH = "./NSW_experiments/"

# State in which the experiment was conducted
STATE = "NSW"

# Names of the PV and no-PV experimental predictions
PV_FILE = "Jan2021_pv_tft_predictions.csv"
NO_PV_FILE = "Jan2021_no_pv_tft_predictions.csv"
#NO_PV_FILE = "test.csv"

In [ ]:
pv.head()
no_pv.head()

In [ ]:
# Load the actuals data (ie: the master v1.0 or v2.0 .CSV dataset)
actual = pd.read_csv(DATA_PATH+ACTUALS_FILE)
actual = actual[actual['STATE'] == STATE]
actual['DATETIME'] = pd.to_datetime(actual['DATETIME'])

# Load experimental forecasts for PV and non-PV models in DATETIME  TOTALDEMAND format
pv = pd.read_csv(EXPERIMENT_PATH+PV_FILE)
no_pv = pd.read_csv(EXPERIMENT_PATH+NO_PV_FILE)

pv.DATETIME = pd.to_datetime(pv.DATETIME)
no_pv.DATETIME = pd.to_datetime(no_pv.DATETIME, )

# Convert Pandas dataframes to darts TimeSeries for analysis
pv_series = TimeSeries.from_dataframe(pv,time_col = 'DATETIME', value_cols='TOTALDEMAND')
no_pv_series = TimeSeries.from_dataframe(no_pv,time_col = 'DATETIME', value_cols='TOTALDEMAND')
actual_series = TimeSeries.from_dataframe(actual, time_col='DATETIME',value_cols=['TOTALDEMAND','TEMPERATURE','FORECASTDEMAND','POWER'])
actual_totaldemand = TimeSeries.from_dataframe(actual, time_col='DATETIME',value_cols='TOTALDEMAND')

# Slice out the subset of the actuals that correspond to the experimental sets
ref_series = actual_series.slice_intersect(pv_series)

In [ ]:
print("PV series",pv_series.start_time(), pv_series.end_time())
print("No PV series",no_pv_series.start_time(), no_pv_series.end_time())
print("Reference series",ref_series.start_time(), ref_series.end_time())

## For the given time range, generate a naive prediction forecast as a baseline reference

In [ ]:
def naive_forecast(train_data, forecast_index, lag=48):
    # For a supplied darts TimeSeries (train_data), returns the day-ago value (ie: lagging by 48 periods)
    #print(forecast_index)
    idx = forecast_index-lag
    #print(idx)
    #print(train_data[idx])

    naive_model = NaiveSeasonal(K=48)
    naive_model.fit(train_data[idx:forecast_index])
    forecast = naive_model.predict(48)

    return forecast
    

In [ ]:
# Generate a naive forecast over the equivalent ranges of the supplied experimental forecasts
start_idx = actual_totaldemand.get_index_at_point(ref_series.start_time())
end_idx = actual_totaldemand.get_index_at_point(ref_series.end_time())

naive_forecasts = naive_forecast(actual_totaldemand, start_idx)
forecast = None

#print(naive_forecasts.start_time(), naive_forecasts.end_time())
for idx in range(start_idx+48, end_idx,48):
    forecast = naive_forecast(actual_totaldemand, idx)
    naive_forecasts = naive_forecasts.concatenate(forecast, ignore_time_axes=True)
    #print(forecast.start_time(), forecast.end_time())

In [ ]:
plt.figure(figsize=(30,10))
ref_series['TOTALDEMAND'].plot(label='Actual')
ref_series['FORECASTDEMAND'].plot(label='AEMO')
pv_series.plot(label='PV')
naive_forecasts.plot(label='naive')
no_pv_series.plot(label='No PV')

In [ ]:
print("AEMO MAPE/RMSE:", mape(ref_series['TOTALDEMAND'], ref_series['FORECASTDEMAND']), rmse(ref_series['TOTALDEMAND'], ref_series['FORECASTDEMAND']))
print("Naive MAPE/RMSE:", mape(ref_series['TOTALDEMAND'], naive_forecasts), rmse(ref_series['TOTALDEMAND'], naive_forecasts))
print("PV prediction MAPE/RMSE", mape(ref_series['TOTALDEMAND'],pv_series), rmse(ref_series['TOTALDEMAND'],pv_series))
print("Non-PV prediction MAPE/RMSE", mape(ref_series['TOTALDEMAND'],no_pv_series), rmse(ref_series['TOTALDEMAND'],no_pv_series))



Calculate the residuals between the predictions and the actuals


In [ ]:
pv_diffs = ref_series['TOTALDEMAND'] - pv_series
non_pv_diffs = ref_series['TOTALDEMAND'] - no_pv_series

## Histogram of the distribution of errors for PV and non-PV roll-forward forecasts

In [ ]:
plot_hist(pv_diffs, bins=20, title="PV-aware prediction residual histogram")
plot_hist(non_pv_diffs, bins=20, title="Non-PV aware prediction residual histogram")

In [ ]:
plot_pacf(pv_diffs, alpha=0.05)
plot_pacf(non_pv_diffs, alpha=0.05)

In [ ]:
plot_residuals_analysis(pv_diffs)
plot_residuals_analysis(non_pv_diffs)

## Correlation of residuals with other predictors: PV data

In [ ]:
def resid_correlation(residuals, main_data):
    cor_df = residuals.pd_dataframe()
    cor_df.rename(columns={'TOTALDEMAND':'RESIDUAL'}, inplace=True)
    return cor_df.merge(main_data, how='left', on='DATETIME')[['RESIDUAL','TOTALDEMAND','POWER','TEMPERATURE']].corr(method='pearson')

In [ ]:
resid_correlation(pv_diffs, actual)

## Correlation of residuals with other predictors: non-PV data

In [ ]:
resid_correlation(non_pv_diffs, actual)

Distribution of errors by time of day

In [ ]:

daily_diffs.head()
#daily_diffs['DATETIME'] = pd.to_datetime(daily_diffs.index)
#daily_diffs.head()

In [ ]:
def plot_residuals_by_hour(residuals):
    res = residuals.pd_dataframe()
    res.head()
    res.rename(columns={'TOTALDEMAND':'Residuals'},inplace=True)
    res['Time of day'] = res.index.time

    sns.set(rc={"figure.figsize": (25,5)})
    errorplot = sns.boxplot(data=res, x='Time of day',y='Residuals')
    errorplot.set_xticklabels(labels=res['Time of day'].unique(), rotation=45)
    ax1 = errorplot.axes
    ax1.axhline(0, ls='--', color='red')    

In [ ]:


plot_residuals_by_hour(pv_diffs)

In [ ]:
plot_residuals_by_hour(non_pv_diffs)